In [1]:
!pip install langchain
!pip install -U langchain-core
!pip install langchain_community
!pip install langchain_experimental
!pip install -U langchain-huggingface
!pip install PyPDF2
!pip install faiss-cpu
!pip install -U langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.17
    Uninstalling langchain-core-0.3.17:
      Successfully uninstalled langchain-core-0.3.17
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 3.2 MB/s eta 0:00:00


In [3]:
!pip install langchain_together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.0 MB/s eta 0:00:00


In [5]:
import os
import PyPDF2
from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_together import ChatTogether

# choose from our 50+ models here: https://docs.together.ai/docs/inference-models
PROJECT_ABSOLUTE_PATH = ''
class PDFRetrievalAgent:
    def __init__(self, name, main_agent, pdf_name="tetgewer", model="Llama-70b", ):
        # sub_agent name
        self.name = name
        self.state = "initial_subagent_state"
        self.main_agent = main_agent
        self.api_key = ""
        self.pdf_name = pdf_name
        self.pdf_path = ""
        self.model = model
        self.pdf_text = ""
        self.texts = None
        self.embeddings = None
        self.vectorstore = None
        self.retrieval_qa_chain = None
        self.llm = None
        # Run the initialization steps
        self.__initialize_agent()

    def __initialize_agent(self):
        self.pdf_path = PROJECT_ABSOLUTE_PATH + '{}.pdf'.format(self.pdf_name)
        self.pdf_text = self.read_pdf(self.pdf_path)
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        self.texts = text_splitter.split_text(self.pdf_text)
        #self.api_key = keys[0]
        self.embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
        self.vectorstore = FAISS.from_texts(self.texts, self.embeddings)
        self.llm = ChatTogether(
              together_api_key="TOGETHER_API_KEY_TO_INSERT_HERE",
              model="meta-llama/Meta-Llama-3-70B-Instruct-Turbo",
        )
        self.retrieval_qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.vectorstore.as_retriever()
        )

    def read_pdf(self, file_path):
        pdf_text = ""
        with open(file_path, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                pdf_text += page.extract_text()
        return pdf_text

    def rag_qa(self, input_query):
        res_ = self.retrieval_qa_chain.run(input_query)
        return res_


In [8]:
pdfRetrievalAgent = PDFRetrievalAgent("RAG Agent", None)
query = """Question: Эрэгтэй хүн хэдэн наснаас тэтгэвэрт гарч болох вэ?

Note: Your answer MUST be in Mongolian.

Your answer:
"""
print("Answer:", pdfRetrievalAgent.rag_qa(query))

Answer: 1957 он ба түүнээс өмнө төрсөн эрэгтэй даатгуулагчийн хувьд 60, 1958 онд төрсөн эрэгтэй даатгуулагчийн хувьд 60 нас 3 сар, 1977 он ба түүнээс хойш төрсөн эрэгтэй даатгуулагчийн хувьд 65 нас болно.
